In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# DifferentialEvolutionDriver

```{note}
DifferentialEvolutionDriver is based on SimpleGADriver and supports most of the same options and capabilities.
```

This [differential evolution](https://en.wikipedia.org/wiki/Differential_evolution) variant of a genetic algorithm optimizer supports only continuous variables. The DifferentialEvolutionDriver supports both constrained and unconstrained optimization.

The DifferentialEvolutionDriver has advantages and disadvantages when compared to the SimpleGADriver:

 - Pros
    - DifferentialEvolutionDriver is typically about 3 times faster than SimpleGADriver
    - DifferentialEvolutionDriver is usually more accurate than SimpleGADriver because it does not limit the number of bits available to represent inputs
    - DifferentialEvolutionDriver does not require the user to manually specify a number of representation bits

 - Cons
    - DifferentialEvolutionDriver only supports continuous input variables; SimpleGADriver also supports discrete
    - DifferentialEvolutionDriver does not support SimpleGADriver’s “compute_pareto” option for multi-objective optimization

Genetic algorithms do not use gradient information to find optimal solutions. This makes them ideal for problems that do not have gradients or problems with many local minima where gradient information is not helpful in finding the global minimum. A well known example of this is finding the global minimum of of the Rastrigin function: ![2D Rastrigin Example](rastrigin2d.png)

The example below shows an OpenMDAO solution of a higher order [Rastrigin function](https://en.wikipedia.org/wiki/Rastrigin_function).

In [2]:
import openmdao.api as om
import numpy as np

ORDER = 6  # dimension of problem
span = 5   # upper and lower limits

class RastriginComp(om.ExplicitComponent):
    def setup(self):
        self.add_input('x', np.zeros(ORDER))
        self.add_output('y', 0.0)

    def compute(self, inputs, outputs):
        x = inputs['x']

        # nth dimensional Rastrigin function, array input and scalar output
        # global minimum at f(0,0,0...) = 0
        n = len(x)
        s = 10 * n
        for i in range(n):
            if np.abs(x[i]) < 1e-200:  # avoid underflow runtime warnings from squaring tiny numbers
                x[i] = 0.0
            s += x[i] * x[i] - 10 * np.cos(2 * np.pi * x[i])

        outputs['y'] = s

prob = om.Problem()

prob.model.add_subsystem('rastrigin', RastriginComp(), promotes_inputs=['x'])
prob.model.add_design_var('x',
                          lower=-span * np.ones(ORDER),
                          upper=span * np.ones(ORDER))
prob.model.add_objective('rastrigin.y')

prob.driver = om.DifferentialEvolutionDriver()
prob.driver.options['max_gen'] = 400
prob.driver.options['Pc'] = 0.5
prob.driver.options['F'] = 0.5

prob.setup()
prob.run_driver()

print(prob['rastrigin.y'])
print(prob['x'])

/Users/leena/dev/eecs481/hw6/OpenMDAO/openmdao/core/group.py:1098: DerivativesWarning:Constraints or objectives [rastrigin.y] cannot be impacted by the design variables of the problem because no partials were defined for them in their parent component(s).


[0.]
[0. 0. 0. 0. 0. 0.]


In [3]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob['rastrigin.y'], 0.0, 1e-6)
assert_near_equal(prob['x'], np.zeros(ORDER), 1e-6)

0.0

## DifferentialEvolutionDriver Options

In [4]:
om.show_options_table("openmdao.drivers.differential_evolution_driver.DifferentialEvolutionDriver")

Option,Default,Acceptable Values,Acceptable Types,Description
F,0.9,N/A,N/A,Differential rate.
Pc,0.9,N/A,N/A,Crossover probability.
debug_print,[],"['desvars', 'nl_cons', 'ln_cons', 'objs', 'totals']",['list'],List of what type of Driver variables to print at each iteration.
invalid_desvar_behavior,warn,"['warn', 'raise', 'ignore']",N/A,Behavior of driver if the initial value of a design variable exceeds its bounds. The default value may beset using the `OPENMDAO_INVALID_DESVAR_BEHAVIOR` environment variable to one of the valid options.
max_gen,100,N/A,N/A,Number of generations before termination.
multi_obj_exponent,1.0,N/A,N/A,Multi-objective weighting exponent.
multi_obj_weights,{},N/A,['dict'],"Weights of objectives for multi-objective optimization.Weights are specified as a dictionary with the absolute namesof the objectives. The same weights for all objectives are assumed, if not given."
penalty_exponent,1.0,N/A,N/A,Penalty function exponent.
penalty_parameter,10.0,N/A,N/A,Penalty function parameter.
pop_size,0,N/A,N/A,Number of points in the GA. Set to 0 and it will be computed as 20 times the total number of inputs.


## DifferentialEvolutionDriver Constructor

The call signature for the DifferentialEvolutionDriver constructor is:

```{eval-rst}
    .. automethod:: openmdao.drivers.differential_evolution_driver.DifferentialEvolutionDriver.__init__
       :noindex:
``` 

## Using DifferentialEvolutionDriver

You can change the number of generations to run the genetic algorithm by setting the “max_gen” option.

In [5]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src15", get_code("openmdao.test_suite.components.branin.Branin"), display=False)

:::{Admonition} `Branin` class definition 
:class: dropdown

{glue:}`code_src15`
:::

In [6]:
import openmdao.api as om
from openmdao.test_suite.components.branin import Branin

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Branin(), promotes_inputs=[('x0', 'xI'), ('x1', 'xC')])

model.add_design_var('xI', lower=-5.0, upper=10.0)
model.add_design_var('xC', lower=0.0, upper=15.0)
model.add_objective('comp.f')

prob.driver = om.DifferentialEvolutionDriver()
prob.driver.options['max_gen'] = 5

prob.setup()
prob.run_driver()

False

You can change the population size by setting the “pop_size” option. The default value for pop_size is 0, which means that the driver automatically computes a population size that is 20 times the total number of input variables.

In [7]:
import openmdao.api as om
from openmdao.test_suite.components.branin import Branin

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Branin(), promotes_inputs=[('x0', 'xI'), ('x1', 'xC')])

model.add_design_var('xI', lower=-5.0, upper=10.0)
model.add_design_var('xC', lower=0.0, upper=15.0)
model.add_objective('comp.f')

prob.driver = om.DifferentialEvolutionDriver()
prob.driver.options['pop_size'] = 10

prob.setup()
prob.run_driver()

False